In [1]:
# %pip install prophet

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from prophet import Prophet
from tqdm import tqdm
import logging

In [2]:
trans = pd.read_csv('Trans_dim.csv')
customers = pd.read_csv('customer_dim.csv', encoding="ISO-8859-1")
time = pd.read_csv('time_dim.csv')
fact = pd.read_csv('fact_table.csv')
store = pd.read_csv('store_dim.csv')
items = pd.read_csv('item_dim.csv', encoding="ISO-8859-1")
tables = [trans, customers, time, fact, store, items]

In [3]:
customers.rename(columns={'coustomer_key': 'customer_key'}, inplace=True)
fact.rename(columns={'coustomer_key': 'customer_key'}, inplace=True)

In [4]:
items.loc[items['unit'] == 'ct.', 'unit'] = 'ct'
items['unit'] = items['unit'].str.lower().str.strip()
items.loc[items['unit'].isna(), 'unit'] = 'box'

time['date'] = pd.to_datetime(time['date'], format='%d-%m-%Y %H:%M')

In [5]:
df = fact.merge(items, on='item_key', how='left').drop(columns=['unit_y', 'unit_price_y']).rename(columns={'unit_x': 'unit', 'unit_price_x': 'unit_price'})
df = df.merge(time, on='time_key', how='left')
df = df.merge(store, on='store_key', how='left')
df.head()

,payment_key,customer_key,time_key,item_key,store_key,quantity,unit,unit_price,total_price,item_name,...,date,hour,day,week,month,quarter,year,division,district,upazila
0,P026,C004510,T049189,I00177,S00307,1,ct,35.0,35.0,M&M Peanut Candy 1.7 oz,...,2016-04-26 17:13:00,17,26,4th Week,4,Q2,2016,DHAKA,JAMALPUR,ISLAMPUR
1,P022,C008967,T041209,I00248,S00595,1,rolls,26.0,26.0,Charmin Ultra Bath Tissue,...,2018-06-11 19:08:00,19,11,2nd Week,6,Q2,2018,DHAKA,DHAKA,CANTONMENT
2,P030,C007261,T03633,I00195,S00496,8,ct,12.5,100.0,Dole Fruit in Gel Cups 4.3 oz,...,2016-11-19 08:19:00,8,19,3rd Week,11,Q4,2016,CHITTAGONG,NOAKHALI,BEGUMGANJ
3,P032,C007048,T084631,I00131,S0086,8,ct,14.0,112.0,Paper Bowls 20 oz Ultra Strong,...,2020-02-01 06:00:00,6,1,1st Week,2,Q1,2020,RANGPUR,RANGPUR,KAUNIA
4,P014,C006430,T071276,I00050,S00488,8,cans,8.0,64.0,Waterloo Sparkling Watermelon 12oz,...,2014-07-15 02:58:00,2,15,3rd Week,7,Q3,2014,CHITTAGONG,KHAGRACHHARI,MATIRANGA


Training & Testing Data

- Take out 2014 because of outliers.
- Exclude covid years vs include covid years (2020+)
  
|Dataset|Train|Test|
|:---|---|---|
|Normal|2015-2017|2018-2019|
|Covid|2015-2019|2020-2021|

In [6]:
normal = df[(2015 <= df['date'].dt.year) & (df['date'].dt.year <= 2019)]
covid = df.copy()

In [7]:
def prepare_data(df):
    df = df.copy()
    df = df.groupby(['year','month', 'division', 'item_name'])[['quantity', 'total_price']].sum().reset_index()
    dates = df['year'].astype(str) + '-' + df['month'].astype(str) + '-01'
    df['date'] = pd.to_datetime(dates, format='%Y-%m-%d')
    df = df.rename(columns={'date': 'ds', 'quantity': 'y'})
    return df

In [8]:
normal = prepare_data(normal)
normal.head()

,year,month,division,item_name,y,total_price,ds
0,2015,1,BARISAL,100% Juice Box Variety 6.75 oz,4,60.00,2015-01-01
1,2015,1,BARISAL,A&W Root Beer - 12 oz cans,9,103.50,2015-01-01
2,2015,1,BARISAL,A&W Root Beer Diet - 12 oz cans,9,60.75,2015-01-01
3,2015,1,BARISAL,Advil 2 pill packets,6,84.00,2015-01-01
4,2015,1,BARISAL,Altoids Small Packs Variety,18,216.00,2015-01-01


In [9]:
covid = prepare_data(covid)
covid.head()

,year,month,division,item_name,y,total_price,ds
0,2014,1,BARISAL,100% Juice Box Variety 6.75 oz,6,90.00,2014-01-01
1,2014,1,BARISAL,A&W Root Beer - 12 oz cans,8,92.00,2014-01-01
2,2014,1,BARISAL,A&W Root Beer Diet - 12 oz cans,3,20.25,2014-01-01
3,2014,1,BARISAL,All Purpose Cleaner 409 Spray,3,54.00,2014-01-01
4,2014,1,BARISAL,Altoids Small Packs Variety,2,24.00,2014-01-01


In [10]:
normal_train = normal[(2015 <= normal['year']) & (normal['year'] <= 2017)]
normal_test = normal[(2018 <= normal['year']) & (normal['year'] <= 2019)]
covid_train = covid[(2015 <= covid['year']) & (covid['year'] <= 2019)]
covid_test = covid[(2019 <= covid['year']) & (covid['year'] <= 2021)]

In [11]:
normal_train.head()

,year,month,division,item_name,y,total_price,ds
0,2015,1,BARISAL,100% Juice Box Variety 6.75 oz,4,60.00,2015-01-01
1,2015,1,BARISAL,A&W Root Beer - 12 oz cans,9,103.50,2015-01-01
2,2015,1,BARISAL,A&W Root Beer Diet - 12 oz cans,9,60.75,2015-01-01
3,2015,1,BARISAL,Advil 2 pill packets,6,84.00,2015-01-01
4,2015,1,BARISAL,Altoids Small Packs Variety,18,216.00,2015-01-01


In [12]:
normal_train.tail()

,year,month,division,item_name,y,total_price,ds
63592,2017,12,SYLHET,Waterloo Sparkling Watermelon 12oz,31,248.0,2017-12-01
63593,2017,12,SYLHET,Welch's 100% Juice Variety10 oz,17,289.0,2017-12-01
63594,2017,12,SYLHET,Wint O Green LifeSavers Sugar Free,13,208.0,2017-12-01
63595,2017,12,SYLHET,Yoplait Original Yogurt Variety Pack,28,336.0,2017-12-01
63596,2017,12,SYLHET,Zoo Animal Cookies/Crackers,6,84.0,2017-12-01


### Model Details

Inputs:
- Time (weekly)
- Division

Output:
- total_price

In [13]:
def predict(division, item_name, train, **kwargs):
    '''
    Input: division, item_name, train, test=None, year=1
    '''
    train = train.copy()
    train = train[(train['division'] == division) & (train['item_name'] == item_name)]
    
    logging.getLogger("prophet").setLevel(logging.WARNING)
    logging.getLogger("cmdstanpy").disabled=True
        
    model = Prophet(
        seasonality_mode='multiplicative',
        changepoint_prior_scale=0.05,
        seasonality_prior_scale=10,
        yearly_seasonality=True,
        weekly_seasonality=False,
        daily_seasonality=False
        )
    model.add_country_holidays(country_name='BD')
    model.fit(train)
    
    if 'year' not in list(kwargs.keys()):
        kwargs['year'] = 1
    
    if 'test' not in list(kwargs.keys()):
        test = model.make_future_dataframe(periods=(12*kwargs['year']), freq='MS')
        test = test[test['ds'].dt.year > train['ds'].dt.year.max()]  
    else:
       test = kwargs['test'].copy()
       test = test[(test['division'] == division) & (test['item_name'] == item_name)]

    prediction = model.predict(test)
    #model.plot(prediction)
    #model.plot_components(prediction)
    
    return prediction[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

def plot(df):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['ds'], y=df['yhat'], mode='lines', name='Prediction', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=df['ds'], y=df['yhat_lower'], mode='lines', name='Lower Bound', line=dict(width=0), fill='tonexty', fillcolor='rgba(102, 178, 255, 0.2)'))
    fig.add_trace(go.Scatter(x=df['ds'], y=df['yhat_upper'], mode='lines', name='Upper Bound', line=dict(width=0), fill='tonexty', fillcolor='rgba(255, 102, 102, 0.2)'))
    fig.update_layout(title='Prediction with Upper and Lower Bounds', xaxis_title='Date', yaxis_title='Total Quantity', legend_title='Legend')
    return fig

In [23]:
result = predict('DHAKA', 'Altoids Small Packs Variety', normal_train, test=normal_test)
result.head()

,ds,yhat,yhat_lower,yhat_upper
0,2018-01-01,85.448431,63.898405,106.106159
1,2018-02-01,119.538412,99.370541,142.535671
2,2018-03-01,101.371868,79.198909,124.147727
3,2018-04-01,89.516647,68.316280,109.525081
4,2018-05-01,125.376498,103.477139,146.779497


In [24]:
normal_test[(normal_test['division'] == 'DHAKA') & (normal_test['item_name'] == 'Altoids Small Packs Variety')].reset_index()['y'].describe()

count     24.000000
mean     106.875000
std       23.506359
min       53.000000
25%       97.750000
50%      102.000000
75%      123.250000
max      156.000000
Name: y, dtype: float64

In [25]:
rmse = np.sqrt(np.mean((normal_test[(normal_test['division'] == 'DHAKA') & (normal_test['item_name'] == 'Altoids Small Packs Variety')].reset_index()['y'] - result['yhat'])**2))
rmse

34.432859030198635

In [26]:
fig1 = plot(result)
fig1.show()
fig2 = px.line(normal_test[(normal_test['division'] == 'DHAKA') & (normal_test['item_name'] == 'Altoids Small Packs Variety')], x='ds', y='y', title='Actual')
fig2.update_layout(title='Actual', xaxis_title='Date', yaxis_title='Total Quantity')
fig2.show()

In [27]:
future = predict('DHAKA', 'Altoids Small Packs Variety', normal_train, year=1)
future.tail()

,ds,yhat,yhat_lower,yhat_upper
7,2018-08-01,84.009202,62.018432,106.841422
8,2018-09-01,83.104696,62.558148,105.674031
9,2018-10-01,129.713263,109.924639,152.584983
10,2018-11-01,108.576800,84.253975,130.285361
11,2018-12-01,83.715791,60.770747,106.486911


In [19]:
normal.head()

,year,month,division,item_name,y,total_price,ds
0,2015,1,BARISAL,100% Juice Box Variety 6.75 oz,4,60.00,2015-01-01
1,2015,1,BARISAL,A&W Root Beer - 12 oz cans,9,103.50,2015-01-01
2,2015,1,BARISAL,A&W Root Beer Diet - 12 oz cans,9,60.75,2015-01-01
3,2015,1,BARISAL,Advil 2 pill packets,6,84.00,2015-01-01
4,2015,1,BARISAL,Altoids Small Packs Variety,18,216.00,2015-01-01


In [20]:
plot(future)

In [21]:
normal['division'].unique()

array(['BARISAL', 'CHITTAGONG', 'DHAKA', 'KHULNA', 'RAJSHAHI', 'RANGPUR',
       'SYLHET'], dtype=object)

In [22]:
def monthly_top(df, years, top):
    divisions = df['division'].unique()
    items = df['item_name'].unique()
    
    result = pd.DataFrame(columns=['ds', 'yhat', 'division', 'item_name'])
    
    for division in divisions:
        for item in tqdm(items):
            prediction = predict(division, item, df, year=years)
            prediction = prediction[['ds', 'yhat']]
            prediction['division'] = division
            prediction['item_name'] = item
            result = pd.concat([result, prediction])
    
    output = pd.DataFrame(columns=['ds', 'yhat', 'division', 'item_name', 'rank'])
    for division in divisions:
        rank = result[result['division'] == division]
        rank.loc[:, 'rank'] = rank.groupby('ds')['yhat'].rank(method="dense", ascending=False)
        output = pd.concat([output, rank[rank['rank'] <= top]])
            
    return output.sort_values('ds').reset_index(drop=True)

In [411]:
result = monthly_top(normal, 1, 1)
result

100%|██████████| 259/259 [00:59<00:00,  4.32it/s]


,ds,yhat,division,item_name,rank
0,2020-01-01,1378.170445,BARISAL,K Cups Original Donut Shop Med. Roast,1.0
1,2020-02-01,1388.224647,BARISAL,K Cups Original Donut Shop Med. Roast,1.0
2,2020-03-01,1397.630192,BARISAL,K Cups Original Donut Shop Med. Roast,1.0
3,2020-04-01,1407.684394,BARISAL,K Cups Original Donut Shop Med. Roast,1.0
4,2020-05-01,1655.897349,BARISAL,Honey Packets,1.0
5,2020-06-01,1427.468470,BARISAL,K Cups Original Donut Shop Med. Roast,1.0
6,2020-07-01,1437.198343,BARISAL,K Cups Original Donut Shop Med. Roast,1.0
7,2020-08-01,1447.252546,BARISAL,K Cups Original Donut Shop Med. Roast,1.0
8,2020-09-01,1457.306749,BARISAL,K Cups Original Donut Shop Med. Roast,1.0
9,2020-10-01,1467.036622,BARISAL,K Cups Original Donut Shop Med. Roast,1.0


In [409]:
result[result['ds'].dt.month == 7].sort_values('division')

,ds,yhat,division,item_name,rank
44,2020-07-01,1437.198343,BARISAL,K Cups Original Donut Shop Med. Roast,1.0
43,2020-07-01,2845.334263,CHITTAGONG,K Cups Original Donut Shop Med. Roast,1.0
48,2020-07-01,6347.906597,DHAKA,Red Bull 12oz,1.0
46,2020-07-01,2122.988147,KHULNA,K Cups Daily Chef Columbian Supremo,1.0
42,2020-07-01,1906.592468,RAJSHAHI,K Cups Daily Chef Columbian Supremo,1.0
45,2020-07-01,1261.337299,RANGPUR,K Cups Daily Chef Columbian Supremo,1.0
47,2020-07-01,912.115889,SYLHET,K Cups Original Donut Shop Med. Roast,1.0
